## Configuracion de Entorno

In [17]:
import os

# Si estás en Jupyter Notebook, anclás a una ruta conocida o fija:
project_root = r"C:\Users\juanm\GitHub\UA_MDM_Labo2_G9"  # ⚠️ Personalizá esto

# Alternativa: usar una carpeta conocida dentro del proyecto
# project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))  # Menos frágil

# Rutas de trabajo
workDir = os.path.join(project_root, "work")
optunaArtifactDir = os.path.join(workDir, "optuna_artifacts")
optunaTempDir = os.path.join(workDir, "optuna_temp_artifacts")

# Crear carpetas si no existen
os.makedirs(optunaArtifactDir, exist_ok=True)
os.makedirs(optunaTempDir, exist_ok=True)

# Cambiar al directorio de trabajo solo si es necesario
if os.getcwd() != workDir:
    os.chdir(workDir)

print("Directorio actual:", os.getcwd())


Directorio actual: C:\Users\juanm\GitHub\UA_MDM_Labo2_G9\work


In [18]:
#Import de librerias basicas tablas y matrices
import numpy as np 
import pandas as pd 

#Gradient Boosting
import lightgbm as lgb

#Funciones auxiliares sklearn
from sklearn.model_selection import train_test_split, StratifiedKFold #Split y cross Validation
from sklearn.metrics import cohen_kappa_score, accuracy_score, balanced_accuracy_score #Metricas
from sklearn.utils import shuffle 

#Visualizacióon
from plotly import express as px

#Plot de matriz de confusion normalizada en actuals
from utiles import plot_confusion_matrix

import os

#Optimizacion de hiperparametros
import optuna
from optuna.artifacts import FileSystemArtifactStore, upload_artifact

#Guardado de objetos en archivos joblib
from joblib import load, dump


In [19]:
# Paths para acceso archivos
#Este notebook asume la siguiente estructura de carpetas a partir de la ubicacion de base_dir 
#(dos niveles arriba de la cƒarpeta donde se ejecuta el notebook). 
# /ƒ/ƒ
# /UA_MDM_Labo2/inputƒ
# /UA_MDM_Labo2/input/petfinder-adoption-prediction/            <- Aca deben ir todos los archivos de datos de la competencia 
# /UA_MDM_Labo2/tutoriales/                       <- Aca deben poner los notebooks y scripts que les compartimos
# /UA_MDM_Labo2/work/                             <- Resultados de notebooks iran dentro de esta carpeta en subcarpetas
# /UA_MDM_Labo2/work/models/                     <- Modelos entrenados en archivos joblibs
# /UA_MDM_Labo2/work/optuna_temp_artifacts/      <- Archivos que queremos dejar como artefacto de un trial de optuna (optuna los copiara a la carpeta de abajo)
# /UA_MDM_Labo2/work/optuna_artifacts/           <- Archivos con artefactos que sibimos a optuna

#Subimos dos niveles para quedar en la carpeta que contiene input y UA_MDM_Labo2
BASE_DIR = '../'

#Datos de entrenamiento 
PATH_TO_TRAIN = os.path.join(BASE_DIR, "input/petfinder-adoption-prediction/train/train.csv")

#Salida de modelos entrenados
PATH_TO_MODELS = os.path.join(BASE_DIR, "work/models")

#Artefactos a subir a optuna
PATH_TO_TEMP_FILES = os.path.join(BASE_DIR, "work/optuna_temp_artifacts")

#Artefactos que optuna gestiona
PATH_TO_OPTUNA_ARTIFACTS = os.path.join(BASE_DIR, "work/optuna_artifacts")


SEED = 42 #Semilla de procesos aleatorios (para poder replicar exactamente al volver a correr un modelo)
TEST_SIZE = 0.2 #Facción para train/test= split

## Feature Engineering

### 🔹 1. **Transformaciones directas útiles**
Variables que ya estaban en los datos, pero necesitaban reformularse para ser más informativas o manejables.

| Variable            | Qué hace                                                   | Por qué ayuda                                                  |
|---------------------|------------------------------------------------------------|-----------------------------------------------------------------|
| `HasName`           | Indica si el animal tiene nombre                           | Un animal con nombre puede verse más cercano o adoptable       |
| `MixBreed`          | Indica si es de raza mixta                                 | Las razas puras pueden ser preferidas o evitadas               |
| `TotalColors`       | Cuántos colores distintos tiene el animal                  | Puede influir en la percepción estética                        |
| `Fee_log`           | Log del precio de adopción                                 | Reduce el impacto de valores extremos                          |
| `DescLength`        | Longitud de la descripción del anuncio                     | Anuncios más detallados pueden atraer más                      |
| `IsFree`            | Indica si el animal es gratuito                            | El costo influye en la decisión de adoptar                     |

👉 **Objetivo**: facilitar al modelo patrones lineales y normalizados.

---

### 🔹 2. **Reagrupaciones o codificaciones útiles**
Transforman datos categóricos o numéricos en formas que capturan relaciones importantes.

| Variable             | Qué hace                                                        | Por qué ayuda                                                   |
|----------------------|------------------------------------------------------------------|------------------------------------------------------------------|
| `AgeBins`            | Categorización de edades (ej: cachorro, adulto)                 | Refleja fases vitales con diferentes tasas de adopción           |
| `Type_Breed_Combo`   | Combina tipo de animal y raza                                   | Algunas razas de perros/gatos tienen mejor aceptación            |
| `RescuerCount`       | Cuántos animales publicó ese rescatista                         | Un rescatista muy activo puede tener diferente tasa de éxito     |

👉 **Objetivo**: capturar efectos de grupos que el modelo pueda aprender más fácilmente.

---

### 🔹 3. **Interacciones entre variables**
Estas variables combinan otras para capturar relaciones no lineales o más profundas.

| Variable               | Qué hace                                                             | Por qué ayuda                                                        |
|------------------------|----------------------------------------------------------------------|----------------------------------------------------------------------|
| `Fee_per_Pet`          | Distribuye el costo entre la cantidad de animales                   | Costo individual más preciso para grupos de mascotas                 |
| `Photo_per_Pet`        | Normaliza la cantidad de fotos según cantidad                       | Más fotos por mascota puede reflejar mayor dedicación                |
| `Age_Fee_Ratio`        | Relación entre edad y costo                                          | Puede capturar si hay sobreprecio para ciertas edades                |
| `DescLength_per_Pet`   | Texto por animal (en caso de grupos)                                | Descripciones muy generales pueden indicar poca información útil     |

👉 **Objetivo**: capturar relaciones más sutiles y específicas entre múltiples variables.


In [20]:
# Cargar el dataset
dataset = pd.read_csv(PATH_TO_TRAIN)

# 1. HasName: binaria si el nombre existe
dataset['HasName'] = dataset['Name'].notnull().astype(int)

# 2. MixBreed: binaria si hay mezcla de razas
dataset['MixBreed'] = (dataset['Breed2'] != 0).astype(int)

# 3. TotalColors: cantidad de colores no nulos
dataset['TotalColors'] = dataset[['Color1', 'Color2', 'Color3']].ne(0).sum(axis=1)

# 4. Fee_log: logaritmo de la tarifa + 1 para manejar valores extremos
dataset['Fee_log'] = np.log1p(dataset['Fee'])

# 5. DescLength: longitud de la descripción de texto
dataset['DescLength'] = dataset['Description'].fillna('').apply(len)

# 6. IsFree: binaria si el fee es cero
dataset['IsFree'] = (dataset['Fee'] == 0).astype(int)

# 7. AgeBins: categorizar la edad en intervalos útiles
dataset['AgeBins'] = pd.cut(dataset['Age'], 
                       bins=[-1, 6, 12, 60, 120, np.inf], 
                       labels=['Cachorro', 'Joven', 'Adulto', 'Mayor', 'Anciano'])

# 8. Type_Breed_Combo: combinación tipo + raza principal
dataset['Type_Breed_Combo'] = dataset['Type'].astype(str) + '_' + dataset['Breed1'].astype(str)

# 9. RescuerCount: cuántos animales publicó cada rescatista
rescuer_counts = dataset['RescuerID'].value_counts()
dataset['RescuerCount'] = dataset['RescuerID'].map(rescuer_counts)

# 10. Fee_per_Pet: tarifa dividida por cantidad
dataset['Fee_per_Pet'] = dataset['Fee'] / dataset['Quantity'].replace(0, 1)

# 11. Photo_per_Pet
dataset['Photo_per_Pet'] = dataset['PhotoAmt'] / dataset['Quantity'].replace(0, 1)

# 12. Age_Fee_Ratio: edad dividido tarifa
dataset['Age_Fee_Ratio'] = dataset['Age'] / (dataset['Fee'] + 1)

# 13. DescLength_per_Pet: longitud del texto por mascota
dataset['DescLength_per_Pet'] = dataset['DescLength'] / dataset['Quantity'].replace(0, 1)

# Verificamos
nuevas_columnas = dataset.columns[-13:]
print("Nuevas columnas agregadas:")
for col in nuevas_columnas:
    print(f"- {col}")


Nuevas columnas agregadas:
- HasName
- MixBreed
- TotalColors
- Fee_log
- DescLength
- IsFree
- AgeBins
- Type_Breed_Combo
- RescuerCount
- Fee_per_Pet
- Photo_per_Pet
- Age_Fee_Ratio
- DescLength_per_Pet


In [21]:
# Listado total de columnas
print("Listado completo de columnas:")
for i, col in enumerate(dataset.columns):
    print(f"{i:02d}: {col}")

Listado completo de columnas:
00: Type
01: Name
02: Age
03: Breed1
04: Breed2
05: Gender
06: Color1
07: Color2
08: Color3
09: MaturitySize
10: FurLength
11: Vaccinated
12: Dewormed
13: Sterilized
14: Health
15: Quantity
16: Fee
17: State
18: RescuerID
19: VideoAmt
20: Description
21: PetID
22: PhotoAmt
23: AdoptionSpeed
24: HasName
25: MixBreed
26: TotalColors
27: Fee_log
28: DescLength
29: IsFree
30: AgeBins
31: Type_Breed_Combo
32: RescuerCount
33: Fee_per_Pet
34: Photo_per_Pet
35: Age_Fee_Ratio
36: DescLength_per_Pet


## Modelo con hiperparametros por default

In [22]:
#Separo un 20% para test estratificado opr target
train, test = train_test_split(dataset,
                               test_size = TEST_SIZE,
                               random_state = SEED,
                               stratify = dataset.AdoptionSpeed)

#Armo listas con features de texto y numericas
char_feats = [f for f in dataset.columns if dataset[f].dtype=='O']
numeric_feats = [f for f in dataset.columns if dataset[f].dtype!='O']

#Lista de features numericas
numeric_feats

['Type',
 'Age',
 'Breed1',
 'Breed2',
 'Gender',
 'Color1',
 'Color2',
 'Color3',
 'MaturitySize',
 'FurLength',
 'Vaccinated',
 'Dewormed',
 'Sterilized',
 'Health',
 'Quantity',
 'Fee',
 'State',
 'VideoAmt',
 'PhotoAmt',
 'AdoptionSpeed',
 'HasName',
 'MixBreed',
 'TotalColors',
 'Fee_log',
 'DescLength',
 'IsFree',
 'AgeBins',
 'RescuerCount',
 'Fee_per_Pet',
 'Photo_per_Pet',
 'Age_Fee_Ratio',
 'DescLength_per_Pet']

In [23]:
#Defino features a usar en un primer modelo de prueba
features = numeric_feats.copy()

# Eliminamos 'AdoptionSpeed' si está
if 'AdoptionSpeed' in features:
    features.remove('AdoptionSpeed')

label = 'AdoptionSpeed'

print(features)

['Type', 'Age', 'Breed1', 'Breed2', 'Gender', 'Color1', 'Color2', 'Color3', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed', 'Sterilized', 'Health', 'Quantity', 'Fee', 'State', 'VideoAmt', 'PhotoAmt', 'HasName', 'MixBreed', 'TotalColors', 'Fee_log', 'DescLength', 'IsFree', 'AgeBins', 'RescuerCount', 'Fee_per_Pet', 'Photo_per_Pet', 'Age_Fee_Ratio', 'DescLength_per_Pet']


In [24]:
#Genero dataframes de train y test con sus respectivos targets
X_train = train[features]
y_train = train[label]

X_test = test[features]
y_test = test[label]

#Entreno un modelo inicial sin modificar hiperparametros. Solamente especifico el numero de clases y el tipo de modelo como clasificacoión
lgb_params = params = {
                        'objective': 'multiclass',
                        'num_class': len(y_train.unique())
                        }


#genero el objeto Dataset que debo pasarle a lightgbm para que entrene
lgb_train_dataset = lgb.Dataset(data=X_train,
                                label=y_train)

#entreno el modelo con los parametros por defecto
lgb_model = lgb.train(lgb_params,
                      lgb_train_dataset)

In [25]:
lgb_model.predict(X_test).argmax(axis=1)

array([2, 1, 2, ..., 1, 4, 4], shape=(2999,))

In [26]:
#Obtengo las predicciones sobre el set de test. El modelo me da una lista de probabilidades para cada clase y tomo la clase con mayor probabilidad con la funcion argmax
y_pred = lgb_model.predict(X_test).argmax(axis=1)

#Calculo el Kappa
print(cohen_kappa_score(y_test,y_pred, weights = 'quadratic'))

#Muestro la matriz de confusión
display(plot_confusion_matrix(y_test,y_pred))

0.36016206755429947


## Modelo con optimizacion de hiperparametros train/test

In [27]:
#Funcion que vamos a optimizar. Optuna requiere que usemos el objeto trial para generar los parametros a optimizar
def lgb_objective(trial):
    #PArametros para LightGBM
    lgb_params = {      
                        #PArametros fijos
                        'objective': 'multiclass',
                        'verbosity':-1,
                        'num_class': len(y_train.unique()),
                        #Hiperparametros a optimizar utilizando suggest_float o suggest_int segun el tipo de dato
                        #Se indica el nombre del parametro, valor minimo, valor maximo 
                        #en elgunos casos el parametro log=True para parametros que requieren buscar en esa escala
                        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
                        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
                        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
                        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
                        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
                        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
                        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
                        } 

    #Genero objeto dataset de entrenamiento
    lgb_train_dataset = lgb.Dataset(data=X_train,
                                    label=y_train)

    #ajuste de modelo
    lgb_model = lgb.train(lgb_params,
                        lgb_train_dataset)
    
    #Devuelvo el score en test
    return(cohen_kappa_score(y_test,lgb_model.predict(X_test).argmax(axis=1),
                             weights = 'quadratic'))

In [28]:
#Defino el estudio a optimizar
study = optuna.create_study(direction='maximize', #buscamos maximizar la metrica
                            storage="sqlite:///../work/db.sqlite3",  # Specify the storage URL here.
                            study_name="04 - LGB Multiclass", #nombre del experimento
                            load_if_exists=True) #continuar si ya existe

#Corremos 100 trials para buscar mejores parametros
study.optimize(lgb_objective, n_trials=10)

[I 2025-04-22 15:35:22,886] Using an existing study with name '04 - LGB Multiclass' instead of creating a new one.
[I 2025-04-22 15:35:23,401] Trial 10 finished with value: 0.31048894863483034 and parameters: {'lambda_l1': 8.705786679511833e-07, 'lambda_l2': 1.0486921815468817e-08, 'num_leaves': 2, 'feature_fraction': 0.6018642589970554, 'bagging_fraction': 0.9986458114949642, 'bagging_freq': 1, 'min_child_samples': 41}. Best is trial 3 with value: 0.3736337838571505.
[I 2025-04-22 15:35:29,420] Trial 11 finished with value: 0.3556478226058404 and parameters: {'lambda_l1': 6.940856304530293e-06, 'lambda_l2': 3.455393299579465, 'num_leaves': 245, 'feature_fraction': 0.6512835031628924, 'bagging_fraction': 0.9955571327136521, 'bagging_freq': 3, 'min_child_samples': 6}. Best is trial 3 with value: 0.3736337838571505.
[I 2025-04-22 15:35:33,975] Trial 12 finished with value: 0.34236775456065927 and parameters: {'lambda_l1': 5.1238133996555754e-05, 'lambda_l2': 6.886089616546512, 'num_leave

In [29]:
#Obtenemos mejor resultado
study.best_params

{'lambda_l1': 2.777973730599744e-05,
 'lambda_l2': 3.585742540428564e-06,
 'num_leaves': 28,
 'feature_fraction': 0.6292350617244538,
 'bagging_fraction': 0.8938519549673785,
 'bagging_freq': 2,
 'min_child_samples': 82}

In [30]:
#Vamos a replicar el resultado de la optimizacion reentrenando el modelo con el mejor conjunto de hiperparametros
#Generamos parametros incluyendo los fijos y la mejor solución que encontro optuna
lgb_params =  {      
                        'objective': 'multiclass',
                        'verbosity':-1,
                        'num_class': len(y_train.unique())} | study.best_params

lgb_train_dataset = lgb.Dataset(data=X_train,
                                label=y_train)


#Entreno
lgb_model = lgb.train(lgb_params,
                    lgb_train_dataset)

#Muestro matriz de confusion y kappa
print(cohen_kappa_score(y_test,lgb_model.predict(X_test).argmax(axis=1),
                             weights = 'quadratic'))

display(plot_confusion_matrix(y_test,lgb_model.predict(X_test).argmax(axis=1)))



0.3736337838571505


## Modelo con optimizacion de hiperparametros con 5 Fold Cross Validation

In [31]:
from optuna.artifacts import FileSystemArtifactStore, upload_artifact
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb
import numpy as np
import os
from joblib import dump
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

def lgb_custom_metric_kappa(dy_pred, dy_true):
    metric_name = 'kappa'
    value = cohen_kappa_score(dy_true.get_label(), dy_pred.argmax(axis=1), weights='quadratic')
    is_higher_better = True
    return (metric_name, value, is_higher_better)

def cv_es_lgb_objective(trial):
    
    # Inicio el store de artefactos (archivos) de Optuna
    artifact_store = FileSystemArtifactStore(base_path=PATH_TO_OPTUNA_ARTIFACTS)

    # Hiperparámetros a optimizar
    lgb_params = {
        'objective': 'multiclass',
        'verbosity': -1,
        'num_class': len(y_train.unique()),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'n_jobs': -1
    }
    
    scores_ensemble = np.zeros((len(y_test), len(y_train.unique())))
    score_folds = 0
    n_splits = 5
    skf = StratifiedKFold(n_splits=n_splits)

    for i, (if_index, oof_index) in enumerate(skf.split(X_train, y_train)):

        print("Numero de corrida:",i)
        
        lgb_if_dataset = lgb.Dataset(data=X_train.iloc[if_index],
                                     label=y_train.iloc[if_index],
                                     free_raw_data=False)
        
        lgb_oof_dataset = lgb.Dataset(data=X_train.iloc[oof_index],
                                      label=y_train.iloc[oof_index],
                                      free_raw_data=False)
        
        print("Se entrena el modelo")

        lgb_model = lgb.train(
            lgb_params,
            lgb_if_dataset,
            valid_sets=lgb_oof_dataset,
            callbacks=[lgb.early_stopping(10, verbose=False)],
            feval=lgb_custom_metric_kappa
        )

        print("Se hacen las predicciones con X_test")
        
        scores_ensemble += lgb_model.predict(X_test)

        score_folds += cohen_kappa_score(
            y_train.iloc[oof_index],
            lgb_model.predict(X_train.iloc[oof_index]).argmax(axis=1),
            weights='quadratic'
        ) / n_splits
    
    print("Guarda predicciones")

    # Guardar predicciones sobre test
    predicted_filename = os.path.join(PATH_TO_TEMP_FILES, f'test_{trial.study.study_name}_{trial.number}.joblib')
    predicted_df = test.copy()
    predicted_df['pred'] = [scores_ensemble[p, :] for p in range(scores_ensemble.shape[0])]
    dump(predicted_df, predicted_filename)

    # ✅ Usar keyword arguments en upload_artifact
    upload_artifact(
        study_or_trial=trial,
        file_path=predicted_filename,
        artifact_store=artifact_store
    )

    print("Guarda matriz de confusión")
    
    # Generar nombre de archivo (¡esto es un string, no lo pises!)
    cm_filename = os.path.join(PATH_TO_TEMP_FILES, f'cm_{trial.study.study_name}_{trial.number}.jpg')

    # Crear y guardar matriz de confusión
    cm = confusion_matrix(y_test, scores_ensemble.argmax(axis=1))
    disp = ConfusionMatrixDisplay(cm)
    disp.plot()
    plt.savefig(cm_filename)
    plt.close()

    # Asociar la imagen como artefacto
    upload_artifact(
        study_or_trial=trial,
        file_path=cm_filename,
        artifact_store=artifact_store
    )

    print("Guarda el score en test como métrica auxiliar")
    
    # Guardar el score en test como métrica auxiliar
    test_score = cohen_kappa_score(y_test, scores_ensemble.argmax(axis=1), weights='quadratic')
    trial.set_user_attr("test_score", test_score)

    # Devuelvo el promedio de los scores del CV (objetivo a maximizar)
    return score_folds

In [32]:
import optuna

#Genero estudio
study = optuna.create_study(direction='maximize',
                            storage="sqlite:///../work/db.sqlite3",  # Specify the storage URL here.
                            study_name="04 - LGB Multiclass CV",
                            load_if_exists = True)

#Corro la optimizacion
study.optimize(cv_es_lgb_objective, n_trials=10)

[I 2025-04-22 15:35:54,998] Using an existing study with name '04 - LGB Multiclass CV' instead of creating a new one.


Numero de corrida: 0
Se entrena el modelo
Se hacen las predicciones con X_test
Numero de corrida: 1
Se entrena el modelo
Se hacen las predicciones con X_test
Numero de corrida: 2
Se entrena el modelo
Se hacen las predicciones con X_test
Numero de corrida: 3
Se entrena el modelo
Se hacen las predicciones con X_test
Numero de corrida: 4
Se entrena el modelo
Se hacen las predicciones con X_test
Guarda predicciones
Guarda matriz de confusión


[I 2025-04-22 15:36:05,525] Trial 3 finished with value: 0.35556435002414405 and parameters: {'lambda_l1': 0.020350834893808886, 'lambda_l2': 0.012822392921872532, 'num_leaves': 202, 'feature_fraction': 0.7545441326099549, 'bagging_fraction': 0.8782818195392265, 'bagging_freq': 3, 'min_child_samples': 25}. Best is trial 2 with value: 0.36989948570675646.


Guarda el score en test como métrica auxiliar
Numero de corrida: 0
Se entrena el modelo
Se hacen las predicciones con X_test
Numero de corrida: 1
Se entrena el modelo
Se hacen las predicciones con X_test
Numero de corrida: 2
Se entrena el modelo
Se hacen las predicciones con X_test
Numero de corrida: 3
Se entrena el modelo
Se hacen las predicciones con X_test
Numero de corrida: 4
Se entrena el modelo
Se hacen las predicciones con X_test
Guarda predicciones
Guarda matriz de confusión


[I 2025-04-22 15:36:15,503] Trial 4 finished with value: 0.3708856001925716 and parameters: {'lambda_l1': 1.7043162292576623e-08, 'lambda_l2': 0.00016117990489850203, 'num_leaves': 65, 'feature_fraction': 0.5188945565476404, 'bagging_fraction': 0.7340437896016265, 'bagging_freq': 4, 'min_child_samples': 78}. Best is trial 4 with value: 0.3708856001925716.


...

Guarda el score en test como métrica auxiliar
Numero de corrida: 0
Se entrena el modelo
Se hacen las predicciones con X_test
Numero de corrida: 1
Se entrena el modelo
Se hacen las predicciones con X_test
Numero de corrida: 2
Se entrena el modelo
Se hacen las predicciones con X_test
Numero de corrida: 3
Se entrena el modelo
Se hacen las predicciones con X_test
Numero de corrida: 4
Se entrena el modelo
Se hacen las predicciones con X_test
Guarda predicciones
Guarda matriz de confusión


[I 2025-04-22 15:37:08,897] Trial 11 finished with value: 0.3650033513869485 and parameters: {'lambda_l1': 1.0781976832629682e-08, 'lambda_l2': 4.863801275958158e-06, 'num_leaves': 26, 'feature_fraction': 0.42434194706821105, 'bagging_fraction': 0.8045501927607112, 'bagging_freq': 7, 'min_child_samples': 73}. Best is trial 6 with value: 0.37410137185857706.


Guarda el score en test como métrica auxiliar
Numero de corrida: 0
Se entrena el modelo
Se hacen las predicciones con X_test
Numero de corrida: 1
Se entrena el modelo
Se hacen las predicciones con X_test
Numero de corrida: 2
Se entrena el modelo
Se hacen las predicciones con X_test
Numero de corrida: 3
Se entrena el modelo
Se hacen las predicciones con X_test
Numero de corrida: 4
Se entrena el modelo
Se hacen las predicciones con X_test
Guarda predicciones
Guarda matriz de confusión


[I 2025-04-22 15:37:12,860] Trial 12 finished with value: 0.3656651183217605 and parameters: {'lambda_l1': 1.0743527141860631e-05, 'lambda_l2': 0.004472451532316544, 'num_leaves': 42, 'feature_fraction': 0.5311844758281924, 'bagging_fraction': 0.5826199564691822, 'bagging_freq': 6, 'min_child_samples': 100}. Best is trial 6 with value: 0.37410137185857706.


Guarda el score en test como métrica auxiliar
